In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from gensim import corpora, models
import pyLDAvis.gensim_models as vis
import pyLDAvis


In [25]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')

In [26]:
df_all = pd.read_hdf('outputs/budzety_clean.hd5', key='df_all')

In [27]:
df_all

,rok,miasto,nazwa,WSZYSTKO,tokens,zbiornik,sadzić,estetyczny,lęgowy,ekologia,...,deszczowy,kwitnąć,ogrodowy,pielęgnacja,smog,zieleniec,names,szkola,ulica,oswietlenie
0,2020,GDYNIA,Projektistatus,0,[projektistatus],False,False,False,False,False,...,False,False,False,False,False,False,projektistatus,False,False,False
1,2020,GDYNIA,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,3,"[doposażenie, park, pomiędzy, estakada, kwiatk...",False,False,False,False,False,...,False,False,False,False,False,False,doposażenie park pomiędzy estakada kwiatkowski...,False,True,False
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy O...,0,"[cykliczny, warsztat, senior, dzielnica, obłuż...",False,False,False,False,False,...,False,False,False,False,False,False,cykliczny warsztat senior dzielnica obłuże roz...,False,False,False
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",3,"[ogród, sąsiedzki, dziki, sadź, ul, źródlana]",False,False,False,False,False,...,False,False,False,False,False,False,ogród sąsiedzki dziki sadź ul źródlana,False,True,False
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy...,0,"[montaż, słup, oświetleniowy, chodnik, ul, nis...",False,False,False,False,False,...,False,False,False,False,False,False,montaż słup oświetleniowy chodnik ul niski odc...,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23687,2020,KRAKÓW,Zielony skwer na Zabtociu,1,"[zielony, skwer, zabtociu]",False,False,False,False,False,...,False,False,False,False,False,False,zielony skwer zabtociu,False,False,False
23688,2020,KRAKÓW,Znani Biatopradniczanie dla Mieszkanców,0,"[znany, mieszkanców]",False,False,False,False,False,...,False,False,False,False,False,False,znany mieszkanców,False,False,False
23690,2020,KRAKÓW,ZRÓBMY SKWER PRZY FABRYCZNEJ! Edukacja i kultu...,1,"[zrobić, skwer, fabryczny, edukacja, kultura, ...",False,False,False,False,False,...,False,False,False,False,False,False,zrobić skwer fabryczny edukacja kultura zieleń,False,False,False
23691,2020,KRAKÓW,Zróbmy sobie Park,1,"[zrobić, park]",False,False,False,False,False,...,False,False,False,False,False,False,zrobić park,False,False,False


In [28]:
# Budowanie słownika
vocab = corpora.Dictionary(df_all['tokens'])
corpus = [vocab.doc2bow(doc) for doc in df_all['tokens']]

# Budowanie korpusu opartego na Tfidf
tfidf_corpus = models.TfidfModel(corpus)[corpus]

In [29]:
# Określenie ilości tematów
N_TOPICS = 10

In [30]:
# Budowa modelu LDA
lda_model = models.LdaModel(tfidf_corpus, num_topics=N_TOPICS, id2word=vocab, passes=2,
random_state=0)

In [39]:
# Znalezione tematy
# 0 to szum
x = lda_model.show_topics()
twords={}
for topic,word in x:
    twords[topic]=re.sub('[^A-Za-z ąćęłóśżźĆŁŚŻŹ]+', '', word)

for n, (k,v) in enumerate(twords.items(),0):
    print(f'{n} {twords[k]}', sep='\n')

0 chodnik  ul  remonta  ulica  bezpieczny  pieszy  budowa  warszawa  wzdłuż  droga
1 zabawa  plac  dziecko  warsztat  rodzinny  piknik  sport  koncert  poprawa  szkolny
2 pies  wybieg  wiata  kobieta  samoobrona  pradze  praski  psi  rucho  nasadzenia
3 seniora  festiwal  łąka  mobilny  rodzina  parka  chmurka  trening  targówku  przyjazny
4 zielony  skwer  wypożyczalnia  pragi  przystanek  monitoring  duży  radość  południe  mały
5 bezpłatny  rower  bezpiecznie  ptak  krakowski  warszawski  stojak  dobry  kulturalny  stacja
6 książka  kino  łódź  park  plenerowy  street  sala  lodowisko  letni  workout
7 biblioteka  zakup  kultura  wole  dziecko  podwórko  nowy  kluba  nowość  wesoły
8 drzewo  ursynów  powietrze  przedszkolak  krzew  kurs  park  toaleta  swój  zielony
9 siłownia  boisko  plenerowy  szkoła  ławka  podstawowy  ursynowie  dziecko  sp  doświetlenie


In [45]:
# Udział poszczególnych słów w tworzeniu danego tematu. Widać tylko 10 najpopularniejszych.
x

[(0,
  '0.031*"chodnik" + 0.028*"ul" + 0.022*"remonta" + 0.019*"ulica" + 0.015*"bezpieczny" + 0.013*"pieszy" + 0.012*"budowa" + 0.012*"warszawa" + 0.011*"wzdłuż" + 0.011*"droga"'),
 (1,
  '0.033*"zabawa" + 0.031*"plac" + 0.018*"dziecko" + 0.011*"warsztat" + 0.010*"rodzinny" + 0.009*"piknik" + 0.008*"sport" + 0.008*"koncert" + 0.008*"poprawa" + 0.008*"szkolny"'),
 (2,
  '0.015*"pies" + 0.013*"wybieg" + 0.012*"wiata" + 0.009*"kobieta" + 0.009*"samoobrona" + 0.008*"pradze" + 0.008*"praski" + 0.008*"psi" + 0.007*"rucho" + 0.007*"nasadzenia"'),
 (3,
  '0.013*"seniora" + 0.012*"festiwal" + 0.011*"łąka" + 0.010*"mobilny" + 0.010*"rodzina" + 0.010*"parka" + 0.010*"chmurka" + 0.010*"trening" + 0.009*"targówku" + 0.009*"przyjazny"'),
 (4,
  '0.014*"zielony" + 0.012*"skwer" + 0.009*"wypożyczalnia" + 0.008*"pragi" + 0.008*"przystanek" + 0.008*"monitoring" + 0.008*"duży" + 0.008*"radość" + 0.008*"południe" + 0.008*"mały"'),
 (5,
  '0.014*"bezpłatny" + 0.014*"rower" + 0.011*"bezpiecznie" + 0.011*"pt

In [40]:
# Wizualizacja tematów
vis_data = vis.prepare(lda_model, tfidf_corpus, vocab)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)

In [41]:
# Zapisanie tematów do DataFrame'a
# metoda zapisywania z https://stackoverflow.com/questions/66403628/how-to-change-topic-list-from-gensim-lda-get-document-topics-to-a-dataframe#:~:text=df%5B%27main_topic%27%5D%20%3D%20%5Bint(str(sorted(LDA_model%5Bi%5D%2Creverse%3DTrue%2Ckey%3Dlambda%20x%3A%20x%5B1%5D)%5B0%5D%5B0%5D).zfill(3))%20for%20i%20in%20corpus%5D
df_all['topics'] = [int(str(sorted(lda_model[i],reverse=True,key=lambda x: x[1])[0][0]).zfill(3)) for i in tfidf_corpus]

In [42]:
df_all['topics_desc'] = df_all['topics'].map(twords)

In [43]:
df_all

,rok,miasto,nazwa,WSZYSTKO,tokens,zbiornik,sadzić,estetyczny,lęgowy,ekologia,...,ogrodowy,pielęgnacja,smog,zieleniec,names,szkola,ulica,oswietlenie,topics,topics_desc
0,2020,GDYNIA,Projektistatus,0,[projektistatus],False,False,False,False,False,...,False,False,False,False,projektistatus,False,False,False,8,drzewo ursynów powietrze przedszkolak krze...
1,2020,GDYNIA,Doposażenie parku pomiędzy Estakadą Kwiatkowsk...,3,"[doposażenie, park, pomiędzy, estakada, kwiatk...",False,False,False,False,False,...,False,False,False,False,doposażenie park pomiędzy estakada kwiatkowski...,False,True,False,9,siłownia boisko plenerowy szkoła ławka po...
2,2020,GDYNIA,Cykliczne warsztaty dla seniorów z dzielnicy O...,0,"[cykliczny, warsztat, senior, dzielnica, obłuż...",False,False,False,False,False,...,False,False,False,False,cykliczny warsztat senior dzielnica obłuże roz...,False,False,False,9,siłownia boisko plenerowy szkoła ławka po...
3,2020,GDYNIA,"Ogród Sąsiedzki przy Dzikim Sadzie, ul. Źródlana",3,"[ogród, sąsiedzki, dziki, sadź, ul, źródlana]",False,False,False,False,False,...,False,False,False,False,ogród sąsiedzki dziki sadź ul źródlana,False,True,False,4,zielony skwer wypożyczalnia pragi przystan...
4,2020,GDYNIA,Montaż słupów oświetleniowych na chodniku przy...,0,"[montaż, słup, oświetleniowy, chodnik, ul, nis...",False,False,False,False,False,...,False,False,False,False,montaż słup oświetleniowy chodnik ul niski odc...,False,True,True,4,zielony skwer wypożyczalnia pragi przystan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23687,2020,KRAKÓW,Zielony skwer na Zabtociu,1,"[zielony, skwer, zabtociu]",False,False,False,False,False,...,False,False,False,False,zielony skwer zabtociu,False,False,False,4,zielony skwer wypożyczalnia pragi przystan...
23688,2020,KRAKÓW,Znani Biatopradniczanie dla Mieszkanców,0,"[znany, mieszkanców]",False,False,False,False,False,...,False,False,False,False,znany mieszkanców,False,False,False,6,książka kino łódź park plenerowy street ...
23690,2020,KRAKÓW,ZRÓBMY SKWER PRZY FABRYCZNEJ! Edukacja i kultu...,1,"[zrobić, skwer, fabryczny, edukacja, kultura, ...",False,False,False,False,False,...,False,False,False,False,zrobić skwer fabryczny edukacja kultura zieleń,False,False,False,4,zielony skwer wypożyczalnia pragi przystan...
23691,2020,KRAKÓW,Zróbmy sobie Park,1,"[zrobić, park]",False,False,False,False,False,...,False,False,False,False,zrobić park,False,False,False,2,pies wybieg wiata kobieta samoobrona prad...


In [44]:
df_all.to_hdf('outputs/budzety_clean_topics_lda.h5', key='df_all')